In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/saeed-abdul-rahim/tutorials/master/imblearn/bank-full.csv", sep = ";")

In [5]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
age          45211 non-null int64
job          45211 non-null object
marital      45211 non-null object
education    45211 non-null object
default      45211 non-null object
balance      45211 non-null int64
housing      45211 non-null object
loan         45211 non-null object
contact      45211 non-null object
day          45211 non-null int64
month        45211 non-null object
duration     45211 non-null int64
campaign     45211 non-null int64
pdays        45211 non-null int64
previous     45211 non-null int64
poutcome     45211 non-null object
y            45211 non-null object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [9]:
!pip install imblearn

    100% |████████████████████████████████| 174kB 499kB/s ta 0:00:01
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

Using TensorFlow backend.


In [12]:
bank = df
bank["default"] = bank["default"].map({"no":0,"yes":1})
bank["housing"] = bank["housing"].map({"no":0,"yes":1})
bank["loan"] = bank["loan"].map({"no":0,"yes":1})
bank["y"] = bank["y"].map({"no":0,"yes":1})
bank.education = bank.education.map({"primary": 0, "secondary":1, "tertiary":2})
bank.month = pd.to_datetime(bank.month, format = "%b").dt.month

In [13]:
bank['month'][:3]

0    5
1    5
2    5
Name: month, dtype: int64

In [14]:
bank.isnull().sum()

age             0
job             0
marital         0
education    1857
default         0
balance         0
housing         0
loan            0
contact         0
day             0
month           0
duration        0
campaign        0
pdays           0
previous        0
poutcome        0
y               0
dtype: int64

In [15]:
bank.drop(["poutcome", "contact"], axis = 1, inplace = True)
bank.dropna(inplace = True)
bank = pd.get_dummies(bank, drop_first = True)

bank.y.value_counts()

X = bank.drop("y", axis = 1)
y = bank.y

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify=y)
print(y_train.value_counts())
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

0    28737
1     3778
Name: y, dtype: int64


In [18]:
confusion_matrix(y_test, y_pred) # does this look good?

array([[9428,  152],
       [ 995,  264]])

In [30]:
264 / (264 + 995)

0.2096902303415409

In [19]:
print(accuracy_score(y_test, y_pred))

print(recall_score(y_test, y_pred))

0.8941784297444414
0.2096902303415409


In [26]:
#Let us use SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify=y)

smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)
np.bincount(y_train)

array([28737, 28737])

In [27]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(confusion_matrix(y_test, y_pred))

print(accuracy_score(y_test, y_pred))

print(recall_score(y_test, y_pred))

[[7697 1883]
 [ 273  986]]
0.8010886613156195
0.7831612390786339


In [31]:
#Let us try undersampling with NearMiss
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify=y)

nr = NearMiss()
X_train, y_train = nr.fit_sample(X_train, y_train)
np.bincount(y_train)

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(confusion_matrix(y_test, y_pred))

print(accuracy_score(y_test, y_pred))

print(recall_score(y_test, y_pred))

[[5085 4495]
 [ 177 1082]]
0.5689639265614909
0.8594122319301033
